In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.regression import (LinearRegression, DecisionTreeRegressor,
                                   RandomForestRegressor, GBTRegressor, GeneralizedLinearRegression)
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.feature import VectorAssembler

In [2]:
# Create a Spark session
spark = SparkSession.builder.appName("ModelTesting").getOrCreate()

# Import data from a CSV file
df = spark.read.csv('model.csv', header=True, inferSchema=True, nullValue='null')

# List of feature columns
feature_cols = df.columns
feature_cols.remove('preu_m2')

# Setup VectorAssembler
assembler = VectorAssembler(inputCols=feature_cols, outputCol='features')

# Transform the data
df = assembler.transform(df)

In [3]:
# Define the models
lr = LinearRegression(featuresCol='features', labelCol='preu_m2')
dt = DecisionTreeRegressor(featuresCol='features', labelCol='preu_m2')
rf = RandomForestRegressor(featuresCol='features', labelCol='preu_m2')
gbt = GBTRegressor(featuresCol='features', labelCol='preu_m2')
glr = GeneralizedLinearRegression(featuresCol='features', labelCol='preu_m2')

# Define the hyperparameter grids
lr_paramGrid = ParamGridBuilder().addGrid(lr.regParam, [0.1, 0.01]).build()
dt_paramGrid = ParamGridBuilder().addGrid(dt.maxDepth, [5, 10]).build()
rf_paramGrid = ParamGridBuilder().addGrid(rf.numTrees, [10, 50]).build()
gbt_paramGrid = ParamGridBuilder().addGrid(gbt.maxIter, [10, 50]).build()
glr_paramGrid = ParamGridBuilder().addGrid(glr.regParam, [0.1, 0.01]).build()

# Define the evaluator
evaluator = RegressionEvaluator(predictionCol='prediction', labelCol='preu_m2', metricName='rmse')

# Define the cross-validators
lr_cv = CrossValidator(estimator=lr, estimatorParamMaps=lr_paramGrid, evaluator=evaluator, numFolds=3)
dt_cv = CrossValidator(estimator=dt, estimatorParamMaps=dt_paramGrid, evaluator=evaluator, numFolds=3)
rf_cv = CrossValidator(estimator=rf, estimatorParamMaps=rf_paramGrid, evaluator=evaluator, numFolds=3)
gbt_cv = CrossValidator(estimator=gbt, estimatorParamMaps=gbt_paramGrid, evaluator=evaluator, numFolds=3)
glr_cv = CrossValidator(estimator=glr, estimatorParamMaps=glr_paramGrid, evaluator=evaluator, numFolds=3)

# Split the data into training, validation, and test sets
train_data, val_data, test_data = df.randomSplit([0.6, 0.2, 0.2])

# Train the models and select the best hyperparameters
lr_model = lr_cv.fit(train_data)
dt_model = dt_cv.fit(train_data)
rf_model = rf_cv.fit(train_data)
gbt_model = gbt_cv.fit(train_data)
glr_model = glr_cv.fit(train_data)

# Use the trained models to make predictions with the test data
lr_predictions = lr_model.transform(test_data)
dt_predictions = dt_model.transform(test_data)
rf_predictions = rf_model.transform(test_data)
gbt_predictions = gbt_model.transform(test_data)
glr_predictions = glr_model.transform(test_data)

# Evaluate the models with the test data
lr_rmse = evaluator.evaluate(lr_predictions)
dt_rmse = evaluator.evaluate(dt_predictions)
rf_rmse = evaluator.evaluate(rf_predictions)
gbt_rmse = evaluator.evaluate(gbt_predictions)
glr_rmse = evaluator.evaluate(glr_predictions)

In [4]:
print('Linear Regression RMSE:', lr_rmse)
print('Decision Tree RMSE:', dt_rmse)
print('Random Forest RMSE:', rf_rmse)
print('Gradient Boosted Tree RMSE:', gbt_rmse)
print('Generalized Linear Regression RMSE:', glr_rmse)

Linear Regression RMSE: 0.11087159878819784
Decision Tree RMSE: 0.1417958217880638
Random Forest RMSE: 0.10892708221753178
Gradient Boosted Tree RMSE: 0.11081088314438982
Generalized Linear Regression RMSE: 0.11087159878819784
